In [1]:
import time 
import datetime
import numpy as np
import pandas as pd
import tweepy
from local_config import *

# assign the values accordingly 
consumer_key = cons_tok 
consumer_secret = cons_sec 
access_token = app_tok 
access_token_secret = app_sec
  
# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# set access to user's access key and access secret  
auth.set_access_token(access_token, access_token_secret) 
  
# calling the api  
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [2]:
list1 = ['enter custom list']

In [7]:
from pymongo import MongoClient
import json

client = MongoClient('enter your mongodb url')
db = client.tweepydb

In [8]:
def get_tweets(candidate,api):
    try :
        alltweets = []  
        #make initial request for most recent tweets (200 is the maximum allowed count)

        new_tweets = api.user_timeline(candidate,count=200,tweet_mode="extended")
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #save the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        #keep grabbing tweets until there are no tweets left to grab
        while len(new_tweets) > 0:
            
            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(candidate,count=200,tweet_mode="extended",max_id=oldest)
            
            #save most recent tweets
            alltweets.extend(new_tweets)
            
            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1
            
        return alltweets
    
    except:
        import traceback
        traceback.print_exc()
        print ('Something went wrong, quitting...')
    

In [10]:
for candidate in list1[0:2]:
    try:
        tweetlist = get_tweets(candidate,api)
        for tweet in tweetlist:
            tw = tweet._json
            tw['candidate'] = candidate
            db.tweets.insert_one(tw)      
    except:
        print("exception")
        continue

In [22]:
c.keys()

dict_keys(['_id', 'created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'quoted_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'candidate'])

In [23]:
c['entities'][]

{'hashtags': [{'text': 'ClimateAction', 'indices': [48, 62]}],
 'symbols': [],
 'user_mentions': [],
 'urls': [{'url': 'https://t.co/al4DRrepTl',
   'expanded_url': 'https://twitter.com/airplaneavocado/status/1102363771895308288',
   'display_url': 'twitter.com/airplaneavocad…',
   'indices': [63, 86]}]}